In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
fraud=pd.read_csv("A:/Data Science/assignments/Random forest/Fraud_check.csv")

In [3]:
fraud.rename(columns={'Marital.Status':'Martial', 'Taxable.Income':'Income', 'City.Population':'Population','Work.Experience':'Exp'},inplace=True)

In [4]:
fraud['group']=np.where(fraud['Income']<=30000,'Risky','Good')

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
SO=SMOTE()
from sklearn.ensemble import RandomForestClassifier


In [6]:
lab=LabelEncoder()
Martial=lab.fit_transform(fraud.Martial)
Undergrad=lab.fit_transform(fraud.Undergrad)
Urban=lab.fit_transform(fraud.Urban)

In [7]:
del fraud['Undergrad']
del fraud['Martial']
del fraud['Urban']

In [8]:
fraud['Undergrad']=Undergrad
fraud['Martial']=Martial
fraud['Urban']=Urban

In [12]:
fraud.Undergrad=fraud.Undergrad.astype(object)
fraud.Martial=fraud.Martial.astype(object)
fraud.Urban=fraud.Urban.astype(object)


In [10]:
fraud.columns

Index(['Income', 'Population', 'Exp', 'group', 'Undergrad', 'Martial',
       'Urban'],
      dtype='object')

In [13]:
fraud.head()

,Income,Population,Exp,group,Undergrad,Martial,Urban
0,68833,50047,10,Good,0,2,1
1,33700,134075,18,Good,1,0,1
2,36925,160205,30,Good,0,1,1
3,50190,193264,15,Good,1,2,1
4,81002,27533,28,Good,0,1,0


In [44]:
X=fraud[[ 'Population', 'Exp', 'Undergrad', 'Martial',
       'Urban']]
y=fraud[['group']]

In [25]:
trains=[]
trains2=[]
for i in range(1,20):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)
    X_train,y_train=SO.fit_sample(X_train,y_train)
    model_1=RandomForestClassifier(criterion='entropy',max_depth=10)
    Reg=model_1.fit(X_train,y_train)
    train=model_1.score(X_train,y_train)
    train2=model_1.score(X_test,y_test)
    trains.append(train)
    trains2.append(train2)
    if (i==19):
        print('Accuracy of train model',np.mean(trains))
        print('Accuracy of test model',np.mean(trains2))
   

Accuracy of train model 0.9662333544705136
Accuracy of test model 0.7742690058479532


In [29]:
#Model shows high variance ,Applying Bagging technique  to reduce the variance

In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold

In [95]:
score=[]
Kf=KFold(n_splits=10,shuffle=False)
for train_index ,test_index in Kf.split(X,y):
    X_train ,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train ,y_test=y.iloc[train_index],y.iloc[test_index]
    X_train1,y_train1=SO.fit_sample(X_train,y_train)
    bag=BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=100,min_samples_split=10,oob_score=True,n_jobs=4,criterion='entropy'),n_estimators=10)
    Reg1=bag.fit(X_train1,y_train1)
    s=bag.score(X_test,y_test)
    score.append(s)

In [127]:
np.mean(bag.score(X_train1,y_train1))

0.8841607565011821

In [128]:
np.mean(score)

0.7583333333333334

In [129]:
values=bag.predict(X_test)

In [130]:
from sklearn.metrics import classification_report,confusion_matrix

In [131]:
np.mean(y_test.values==values)

0.76

In [134]:
confusion_matrix(y_test,values)

array([[142,   2],
       [ 26,  10]], dtype=int64)

In [135]:
y_test.group.value_counts()

Good     144
Risky     36
Name: group, dtype: int64

In [ ]:
#Accuracy is 76%